### Notebook to numerically integrate f(x) = e^(-2x)cos(10x)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

## First, define a function for taking an integral

In [ ]:
def func(x):
    a = 2.0
    b = 10.0
    return (np.exp(-1*a*x))*np.cos(b*x)

## Next, define the functions integral 

In [ ]:
def func_integral(x):
    a = 2.0
    b = 10.0
    c = 5.0
    d = 52.0
    return ((np.exp(-1*a*x))*(c*np.sin(b*x) - np.cos(b*x)))/d

## We will beging with the Trapezoid Method

### Define the core of the trapezoid method

In [ ]:
def trapezoid_core(f,x,h):
    return 0.5*h*(f(x+h) + f(x))

### Define a wrapper function to perform the method

In [ ]:
def trapezoid_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    
    #define x values to perform the trapezoid rule
    x = np.linspace(a,b,N)
    h = x[1] - x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using the trapezoid method
    for i in range(0,len(x)-1,1):
        Fint += trapezoid_core(f,x[i],h)
        
    #return the answer
    return Fint

## Now we will use the Simpson's Method

### Define the core of Simpson's Method

In [ ]:
def simpson_core(f,x,h):
    return h*(f(x) + 4*f(x+h) + f(x+2*h))/3.

### Define a wrapper function to perform Simpson's Method

In [ ]:
def simpsons_method(f,a,b,N):
    #f == function to integrate
    #a == lower limit of integration
    #b == upper limit of integration
    #N == number of function evaluations to use
    
    #define x values to perform simpsons rule
    x = np.linspace(a,b,N)
    h = x[1]-x[0]
    
    #define the value of the integral
    Fint = 0.0
    
    #perform the integral using simpsons method
    for i in range(0,len(x)-2,2):
        Fint += simpson_core(f,x[i],h)
        
    #apply simpson's rule over the last interval 
    #if N is even
    if((N%2) == 0):
        Fint += simpson_core(f,x[-2],0.5*h)
        
    return Fint

## Lastly we will use Romberg Integration

### Define the Romberg Integration core

In [ ]:
def romberg_core(f,a,b,i):
    
    #difference between a and b
    h = b-a
    
    #increment between new func evals
    dh = h/2.**(i)
    
    #cofactor
    K = h/2.**(i+1)
    
    #function evaluations
    M = 0.0
    for j in range(2**i):
        M += f(a + 0.5*dh + j*dh)
        
    return K*M

### Define a wrapper function to perform Romberg Integration

In [ ]:
def romberg_integration(f,a,b,tol):
    
    #iteration variable
    i = 0
    
    #max number of iterations
    imax = 1000
    
    #create an error estimate
    #set to a large value
    delta = 100.0*np.fabs(tol)
    
    #set an array of integral answers
    I = np.zeros(imax,dtype=float)
    
    #get the zeroth romberg iteration
    I[0] = 0.5*(b-a)*(f(a) + f(b))
    
    #iterate by 1
    i += 1
    
    while(delta>tol):
        
        #find this romberg iteration
        I[i] = 0.5*I[i-1] + romberg_core(f,a,b,i)
        
        #compute the new fractional error estimate
        delta = np.fabs((I[i] - I[i-1])/I[i])
        
        print(i,I[i],I[i-1],delta)
        
        if(delta>tol):
            
            #iterate 
            i+=1
            
            #if we've reached the maximum iterations
            if(i>imax):
                print("Maximum iterations reached.")
                raise StopIteration('Stopping iterations after ',i)
    #return answer
    return I[i]

## Now to check the integrals

In [ ]:
Answer = func_integral(np.pi) - func_integral(0)
print(Answer)
print("Trapezoid Method")
print(trapezoid_method(func,0,np.pi,3388))
print("Simpson's Method")
print(simpsons_method(func,0,np.pi,141))
print("Romberg Integration")
tolerance = 1.0e-6
RI = romberg_integration(func,0,np.pi,tolerance)
print(RI,(RI-Answer)/Answer,tolerance)

### Romberg Integration takes 26 iterations to reach the specified accuracy
### The Trapezoid Method takes 3388 intervals to reach the specified accuracy
### Simpson's Method takes 141 intervals to reach the specified accuracy